In [1]:
import numpy as np
import pandas as pd
from datetime import datetime
from dateutil.relativedelta import relativedelta
from scipy.optimize import least_squares

In [2]:

# ----------------------------
# A) Utilidades de tiempo
# ----------------------------
VAL_DATE = datetime(2024, 6, 28)
LLP_DATE = datetime(2028, 6, 30)  # fin del tramo líquido TZX

def yearfrac_act365(d0, d1):
    return (d1 - d0).days / 365.0

In [3]:

# ----------------------------
# B) Curva corta (t <= LLP) "conocida"
#    En tu caso, viene de TZX25..TZX28.
#    Acá pongo un ejemplo: DF_liquido(t) = exp(-z*t)
# ----------------------------
def df_short(t):
    # Ejemplo: tasa cero continua real 6% hasta LLP (solo ilustrativo)
    z = 0.06
    return np.exp(-z * t)

t_llp = yearfrac_act365(VAL_DATE, LLP_DATE)
DF_LLP = df_short(t_llp)

In [7]:
#-------------------------
# C) Survival soberano S(t) desde PD Fitch (acá: hazard constante como ejemplo)
#    Reemplazá esto por tu hazard anual derivado de Fitch CCC.
# ----------------------------
LGD = 0.60
LAMBDA = 0.08  # hazard anual constante (ejemplo)

def S(t):
    # Survival en tiempo continuo: S(t)=exp(-λ t)
    return np.exp(-LAMBDA * t)


In [8]:

# ----------------------------
# D) Nelson–Siegel (tasas cero continuas) para la cola
# ----------------------------
def z_ns(t, beta0, beta1, beta2, tau):
    x = t / tau
    # factores NS estándar
    f1 = (1 - np.exp(-x)) / x
    f2 = f1 - np.exp(-x)
    return beta0 + beta1 * f1 + beta2 * f2

def df_ns(t, beta0, beta1, beta2, tau):
    return np.exp(-z_ns(t, beta0, beta1, beta2, tau) * t)

def df_rf(t, beta0, beta1, beta2, tau):
    """
    Curva completa:
      - tramo corto: df_short
      - tramo largo: NS pegado a LLP para continuidad
    """
    if t <= t_llp:
        return df_short(t)
    # "pegar" al LLP:
    return DF_LLP * df_ns(t, beta0, beta1, beta2, tau) / df_ns(t_llp, beta0, beta1, beta2, tau)


In [9]:

# ----------------------------
# E) Cashflows reales (ejemplo bullet semestral)
# ----------------------------
def make_bullet_cashflows(maturity_date, coupon_rate_annual, notional=100.0):
    """
    Crea pagos semestrales desde el próximo semestre hasta maturity (simplificado).
    En la práctica: reemplazar por schedule real (PAR/DISC/CUAP).
    """
    # Genero fechas cada 6 meses
    dates = []
    d = maturity_date
    # retrocedo semestres hasta pasar VAL_DATE
    while d > VAL_DATE:
        dates.append(d)
        d = d - relativedelta(months=6)
    dates = sorted(dates)

    rows = []
    for i, pay_date in enumerate(dates, start=1):
        alpha = 0.5  # semestre
        interest = notional * coupon_rate_annual * alpha
        principal = notional if pay_date == maturity_date else 0.0
        cf = interest + principal
        rows.append((pay_date, cf, notional))  # notional outstanding simplificado (=100)
    return pd.DataFrame(rows, columns=["date", "cf_real", "N_prev"])


In [10]:

# ----------------------------
# F) Precio con crédito (PV esperado)
# ----------------------------
def price_with_credit(cfs: pd.DataFrame, beta0, beta1, beta2, tau):
    pv = 0.0
    t_prev = 0.0
    S_prev = 1.0
    for _, row in cfs.iterrows():
        t = yearfrac_act365(VAL_DATE, row["date"])
        df = df_rf(t, beta0, beta1, beta2, tau)
        S_t = S(t)

        cf = row["cf_real"]
        N_prev = row["N_prev"]

        # pago si sobrevive hasta t
        pv += df * (cf * S_t)

        # recupero si default ocurre en (t_prev, t]
        dS = S_prev - S_t  # prob. de default en el intervalo
        recovery = (1 - LGD) * N_prev
        pv += df * (recovery * dS)

        t_prev = t
        S_prev = S_t

    return pv

In [11]:

# ----------------------------
# G) Datos de bonos cola (PARP/DICP/CUAP) con precios reales "observados" (ejemplo)
#    OJO: acá uso precios inventados en unidad real por 100.
#    En tu caso: P_real = P_nom / CER_hoy (y consistente dirty/clean).
# ----------------------------
bonds = [
    {"name": "DICP", "mat": datetime(2033, 12, 31), "c": 0.0583, "P_real_mkt": 62.0},
    {"name": "PARP", "mat": datetime(2038, 12, 31), "c": 0.0177, "P_real_mkt": 45.0},
    {"name": "CUAP", "mat": datetime(2045, 12, 31), "c": 0.0331, "P_real_mkt": 40.0},
]

cashflows = {b["name"]: make_bullet_cashflows(b["mat"], b["c"]) for b in bonds}


In [17]:
cashflows['CUAP']

,date,cf_real,N_prev
0,2024-06-30,1.655,100.0
1,2024-12-30,1.655,100.0
2,2025-06-30,1.655,100.0
3,2025-12-30,1.655,100.0
4,2026-06-30,1.655,100.0
5,2026-12-30,1.655,100.0
6,2027-06-30,1.655,100.0
7,2027-12-30,1.655,100.0
8,2028-06-30,1.655,100.0
9,2028-12-30,1.655,100.0


In [12]:

# ----------------------------
# H) Calibración NS para la cola (tau fijo para identificabilidad)
# ----------------------------
TAU = 3.0  # años (fijo)

def residuals(params):
    beta0, beta1, beta2 = params
    res = []
    for b in bonds:
        model = price_with_credit(cashflows[b["name"]], beta0, beta1, beta2, TAU)
        res.append(model - b["P_real_mkt"])
    return np.array(res)

# punto inicial razonable
x0 = np.array([0.06, -0.03, 0.02])  # beta0, beta1, beta2

sol = least_squares(residuals, x0, bounds=([-0.05, -1.0, -1.0], [0.30, 1.0, 1.0]))
beta0_hat, beta1_hat, beta2_hat = sol.x

print("NS cola calibrada (tau fijo=3):")
print(beta0_hat, beta1_hat, beta2_hat)

NS cola calibrada (tau fijo=3):
0.04013864781448777 0.9999999999992724 -0.181165717786623


In [15]:

# ----------------------------
# I) Curva final en tus fechas (30-Jun de 2029 a 2045)
# ----------------------------
pay_dates = [datetime(y, 6, 30) for y in range(2029, 2046)]
out = []
for d in pay_dates:
    t = yearfrac_act365(VAL_DATE, d)
    df = df_rf(t, beta0_hat, beta1_hat, beta2_hat, TAU)
    y_eff = df**(-1.0 / t) - 1.0  # tasa efectiva anual equivalente
    out.append((d.date(), t, df, y_eff))

curve = pd.DataFrame(out, columns=["date", "t_years", "DF_rf", "y_eff_annual"])
print(curve)


          date    t_years     DF_rf  y_eff_annual
0   2029-06-30   5.008219  0.641660      0.092637
1   2030-06-30   6.008219  0.553771      0.103366
2   2031-06-30   7.008219  0.496089      0.105199
3   2032-06-30   8.010959  0.455465      0.103150
4   2033-06-30   9.010959  0.425161      0.099567
5   2034-06-30  10.010959  0.401137      0.095538
6   2035-06-30  11.010959  0.381104      0.091564
7   2036-06-30  12.013699  0.363642      0.087849
8   2037-06-30  13.013699  0.348006      0.084490
9   2038-06-30  14.013699  0.333625      0.081483
10  2039-06-30  15.013699  0.320180      0.078807
11  2040-06-30  16.016438  0.307435      0.076422
12  2041-06-30  17.016438  0.295334      0.074306
13  2042-06-30  18.016438  0.283761      0.072417
14  2043-06-30  19.016438  0.272664      0.070726
15  2044-06-30  20.019178  0.261977      0.069200
16  2045-06-30  21.019178  0.251734      0.067826
